In [ ]:
import time
import pandas
import pyodbc
import datetime
import pandas as pd
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from datetime import date, timedelta, datetime

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import udf
from unidecode import unidecode
from pyspark.sql.functions import lpad, regexp_replace, trim, col, concat_ws, collect_list, desc, row_number, monotonically_increasing_id 
from pyspark.sql.window import Window
conf = SparkConf().setMaster("spark://192.168.0.210:7077")
conf.set("spark.executor.memory", "6g")
conf.set("spark.driver.memory", "6g")
conf.set("spark.executor.cores", "4")
conf.set("spark.driver.cores", "4")
conf.set("spark.driver.bindAddress", "0.0.0.0")
conf.set('spark.driver.host','192.168.0.210')
conf.set("spark.driver.port", "33139")
conf.set("spark.driver.blockManager.port", "45029")
conf.set("spark.driver.maxResultSize", "4096mb")
conf.set("spark.default.parallelism", "512")
conf.set("spark.sql.shuffle.partitions", "128")
conf.set("spark.sql.debug.maxToStringFields","200")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.files.maxPartitionBytes", "128mb")
conf.set("spark.shuffle.file.buffer", "1mb")
conf.set("spark.sql.autoBroadcastHashJoin", "12mb")
conf.set("spark.sql.repl.eagerEval.enabled", True)
spark = SparkSession.builder.config(conf=conf.setAppName("Jupyter")).getOrCreate()



drivers = [item for item in pyodbc.drivers()]
driver = drivers[-1]
connection_url = URL.create(
    "mssql+pyodbc",
    username="sa",
    password="10256589Gu@",
    host="mysql",
    port=1433,
    database="data",
    query={
        "driver": driver,
        "TrustServerCertificate": 'yes'
    },
)
engine = create_engine(
    connection_url, connect_args={'ssl_context':True}
)


data_path = ('/home/jovyan/GitHub/spark notebook/data')

def get_spark_df_shape(df):
    print("Number of rows: ", df.count())
    print("Number of columns: ", len(df.columns))
    
def save_df_to_parquet_hdfs(df, name):
    df.columns = df.columns.map(lambda x: unidecode(x))
    df.columns = df.columns.str.replace(' ', '', regex=False)
    spark_df = spark.createDataFrame(df)
    save_spark_df_to_parquet_hdfs(spark_df,name)
    
def save_spark_df_to_parquet_hdfs(df, name):
    df.write.mode('overwrite').partitionBy("spark_partition","Country").parquet(f"hdfs://192.168.0.101:8020/data/{name}.parquet")
    
def read_hdfs_parquet(filename):
    return spark.read.parquet(f"hdfs://192.168.0.101:8020/data/{filename}.parquet")

def spark_to_pd_head(df, n=5):
    display(df.limit(n).toPandas().head(n))

In [ ]:
#save_df_to_parquet_hdfs(pandas.read_csv(os.path.join(data_path,"customers-2000000.csv")), 'customers-2000000')
#get_spark_df_shape(df)
#spark_to_pd_head(df)

In [ ]:
df = read_hdfs_parquet("customers-2000000")
#df = df.union(df)
#df = df.union(df)
#df = df.union(df)
#df = df.union(df)
df = df.drop('Index')
df = df.withColumn("spark_index", monotonically_increasing_id())
df = df.withColumn("spark_partition", col('spark_index').substr(-2, 2).cast('int'))
get_spark_df_shape(df)
spark_to_pd_head(df)

In [ ]:
save_spark_df_to_parquet_hdfs(df,'large_df')
df =  read_hdfs_parquet("large_df")
spark_to_pd_head(df)

In [ ]:
start_time = time.time()

for i in range(0,99):
    
    chunk_start_time = time.time()
    chunk = read_hdfs_parquet("large_df").filter(col('spark_index').substr(-2, 2).cast('int') == i)
    chunk.toPandas().to_sql(name='big_table_test', con=engine, index=False, if_exists='append', schema='dbt')
    chunk_end_time = time.time()
    chunk_elapsed_time = chunk_end_time - chunk_start_time
    print(f"Progress: {(i+1)}% in {chunk_elapsed_time} seconds")

end_time = time.time()
elapsed_time = end_time - start_time
print("Data insertion complete in {elapsed_time} seconds.")

In [ ]:
df = pd.read_sql('SELECT top 1000000 * FROM dbt.big_table_test', engine)
df.shape